In [1]:
import pandas as pd
from urllib import request
import folium
from folium.plugins import TimestampedGeoJson
import json
from geojson import Point, Feature, FeatureCollection, dump
import pickle
from datetime import datetime, date
import os
import h3
from shapely.geometry import Point
import geopandas as gpd
from keplergl import KeplerGl

In [2]:
def get_most_recent_upload_date():
    r = request.urlopen('https://www.nycgovparks.org/tree-work-orders/street_tree_planting.csv')

    last_build_date = r.readlines()[5].decode('utf-8').split(',')[1].strip('\n')

    last_build_date = datetime.strptime(last_build_date.strip('"'), '%Y-%m-%d %H:%M:%S')

    return last_build_date

def download_new_data():

    df = pd.read_csv('https://www.nycgovparks.org/tree-work-orders/street_tree_planting.csv', skiprows=7)
    df['CompletedDate'] = pd.to_datetime(df['CompletedDate'])
    df['PlantingSeason'] = pd.to_datetime(df['PlantingSeason'])

    return df

def load_tree_data(save=True, return_data=True):

    today = datetime.now()
    file_names = os.listdir('../data/')
    today_file = f'../data/street_tree_planting_{today.strftime("%Y_%m_%d")}.pkl'

    if os.path.exists(today_file):
        df = pd.read_pickle(today_file)

    elif not os.path.exists(today_file):
        most_recent_web = get_most_recent_upload_date()
        if most_recent_web < today:
            most_recent_file = f'../data/street_tree_planting_{most_recent_web.strftime("%Y_%m_%d")}.pkl'
            if os.path.exists(most_recent_file):
                df = pd.read_pickle(most_recent_file)
            elif not os.path.exists(most_recent_file):
                df = download_new_data()
                if save:
                    df.to_pickle(f'../data/street_tree_planting_{most_recent_web.strftime("%Y_%m_%d")}.pkl')
        elif most_recent_web == today:
            df = get_most_recent_upload_date()
            if save:
                df.to_pickle(today_file)

    if return_data:
        return df

In [3]:
df = load_tree_data()
df

,lng,lat,Borough,ZipCode,BuildingNumber,StreetName,FiscalYear,PlantingSpaceID,CommunityBoard,PlantingSeason,CityCouncil,TreeID,WOId,WOStatus,CompletedDate
0,-73.709503,40.740340,Queens,11004,82-030,260 STREET,0,148027,413,2022-05-31,23,114029,15525248,Completed,2021-12-07
1,-73.954008,40.814634,Manhattan,10027,464,WEST 129 STREET,0,156427,109,2022-05-31,7,0,16644663,Not Completed,NaT
2,-73.712984,40.735428,Queens,11001,254-04,84 ROAD,0,177638,413,2022-05-31,23,142438,8573730,Completed,2021-12-07
3,-73.952187,40.799678,Manhattan,10026,37,MALCOLM X BOULEVARD,0,178031,110,2022-05-31,9,143233,9138879,Not Completed,NaT
4,-73.876284,40.828165,Bronx,10472,1155,MANOR AVENUE,0,191641,209,2022-05-31,18,0,14343624,Not Completed,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8057,-73.868403,40.862017,Bronx,10467,2418,OLINVILLE AVENUE,0,2445051,211,2022-05-31,15,13125821,14370128,Completed,2022-07-13
8058,-73.785172,40.727778,Queens,11366,183-02,UNION TURNPIKE,0,4309386,408,2022-05-31,24,13133256,12931606,Completed,2022-05-24
8059,-73.990085,40.665143,Brooklyn,11215,568,5 AVENUE,0,3926932,307,2022-05-31,39,13145237,14539255,Completed,2021-12-03
8060,-74.004315,40.649622,Brooklyn,11232,531,41 STREET,0,227912,307,2022-05-31,38,13148613,16783658,Completed,2022-05-13


In [4]:
h3_res = 8
def geo_to_h3(row):
    return h3.geo_to_h3(lat=row.lat,lng=row.lng,resolution = h3_res)

In [5]:
df['hex_id'] = df.apply(geo_to_h3, axis=1)
df

,lng,lat,Borough,ZipCode,BuildingNumber,StreetName,FiscalYear,PlantingSpaceID,CommunityBoard,PlantingSeason,CityCouncil,TreeID,WOId,WOStatus,CompletedDate,hex_id
0,-73.709503,40.740340,Queens,11004,82-030,260 STREET,0,148027,413,2022-05-31,23,114029,15525248,Completed,2021-12-07,882a10042dfffff
1,-73.954008,40.814634,Manhattan,10027,464,WEST 129 STREET,0,156427,109,2022-05-31,7,0,16644663,Not Completed,NaT,882a1008cbfffff
2,-73.712984,40.735428,Queens,11001,254-04,84 ROAD,0,177638,413,2022-05-31,23,142438,8573730,Completed,2021-12-07,882a100425fffff
3,-73.952187,40.799678,Manhattan,10026,37,MALCOLM X BOULEVARD,0,178031,110,2022-05-31,9,143233,9138879,Not Completed,NaT,882a1008d5fffff
4,-73.876284,40.828165,Bronx,10472,1155,MANOR AVENUE,0,191641,209,2022-05-31,18,0,14343624,Not Completed,NaT,882a1001adfffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8057,-73.868403,40.862017,Bronx,10467,2418,OLINVILLE AVENUE,0,2445051,211,2022-05-31,15,13125821,14370128,Completed,2022-07-13,882a100139fffff
8058,-73.785172,40.727778,Queens,11366,183-02,UNION TURNPIKE,0,4309386,408,2022-05-31,24,13133256,12931606,Completed,2022-05-24,882a100ecbfffff
8059,-73.990085,40.665143,Brooklyn,11215,568,5 AVENUE,0,3926932,307,2022-05-31,39,13145237,14539255,Completed,2021-12-03,882a107747fffff
8060,-74.004315,40.649622,Brooklyn,11232,531,41 STREET,0,227912,307,2022-05-31,38,13148613,16783658,Completed,2022-05-13,882a107701fffff


In [6]:
tree_counts_lat_long = df[df['WOStatus'] == 'Completed'].value_counts('hex_id').reset_index().rename({'hex_id': 'Hex ID', 0: 'value'}, axis=1)
tree_counts_lat_long = tree_counts_lat_long.merge(df[df['WOStatus'] == 'Completed'].groupby('hex_id').last()[['lat', 'lng']].reset_index().rename({'hex_id': 'Hex ID'}, axis=1), on='Hex ID')

tree_counts_lat_long

,Hex ID,value,lat,lng
0,882a100157fffff,125,40.868989,-73.840153
1,882a10010dfffff,91,40.884259,-73.854384
2,882a1005b9fffff,63,40.677720,-73.740996
3,882a100dc5fffff,63,40.710596,-73.937014
4,882a107469fffff,63,40.614911,-73.991221
...,...,...,...,...
558,882a107527fffff,1,40.602977,-74.097827
559,882a107501fffff,1,40.609755,-74.063037
560,882a103951fffff,1,40.613430,-73.820614
561,882a1039c9fffff,1,40.584737,-73.813229


In [8]:
map_1 = KeplerGl(height=600)
map_1.add_data(tree_counts_lat_long[['lat', 'lng', 'value']], name='lat_longs')
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'lat_longs': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2…

In [23]:
config = {
                    "version": "v1",
                    "config": {
                        "visState": {
                            "filters": [],
                            "layers": [
                                {
                                    "id": "s3afk3",
                                    "type": "grid",
                                    "config": {
                                        "dataId": "lat_longs",
                                        "label": "Point",
                                        "color": [
                                            221,
                                            178,
                                            124
                                        ],
                                        "highlightColor": [
                                            252,
                                            242,
                                            26,
                                            255
                                        ],
                                        "columns": {
                                            "lat": "lat",
                                            "lng": "lng"
                                        },
                                        "isVisible": True,
                                        "visConfig": {
                                            "opacity": 0.8,
                                            "worldUnitSize": 1,
                                            "colorRange": {
                                                "name": "Uber Viz Diverging 1.5",
                                                "type": "diverging",
                                                "category": "Uber",
                                                "colors": [
                                                    "#00939C",
                                                    "#5DBABF",
                                                    "#BAE1E2",
                                                    "#F8C0AA",
                                                    "#DD7755",
                                                    "#C22E00"
                                                ]
                                            },
                                            "coverage": 1,
                                            "sizeRange": [
                                                0,
                                                500
                                            ],
                                            "percentile": [
                                                0,
                                                100
                                            ],
                                            "elevationPercentile": [
                                                0,
                                                100
                                            ],
                                            "elevationScale": 13,
                                            "enableElevationZoomFactor": True,
                                            "colorAggregation": "average",
                                            "sizeAggregation": "average",
                                            "enable3d": True
                                        },
                                        "hidden": False,
                                        "textLabel": [
                                            {
                                                "field": None,
                                                "color": [
                                                    255,
                                                    255,
                                                    255
                                                ],
                                                "size": 18,
                                                "offset": [
                                                    0,
                                                    0
                                                ],
                                                "anchor": "start",
                                                "alignment": "center"
                                            }
                                        ]
                                    },
                                    "visualChannels": {
                                        "colorField": {
                                            "name": "value",
                                            "type": "integer"
                                        },
                                        "colorScale": "quantile",
                                        "sizeField": {
                                            "name": "value",
                                            "type": "integer"
                                        },
                                        "sizeScale": "linear"
                                    }
                                }
                            ],
                            "interactionConfig": {
                                "tooltip": {
                                    "fieldsToShow": {
                                        "lat_longs": [
                                            {
                                                "name": "value",
                                                "format": None
                                            }
                                        ]
                                    },
                                    "compareMode": False,
                                    "compareType": "absolute",
                                    "enabled": True
                                },
                                "brush": {
                                    "size": 0.5,
                                    "enabled": False
                                },
                                "geocoder": {
                                    "enabled": False
                                },
                                "coordinate": {
                                    "enabled": False
                                }
                            },
                            "layerBlending": "normal",
                            "splitMaps": [],
                            "animationConfig": {
                                "currentTime": None,
                                "speed": 1
                            }
                        },
                        "mapState": {
                            "bearing": 24,
                            "dragRotate": True,
                            "latitude": 40.616940256509345,
                            "longitude": -73.9878453861782,
                            "pitch": 50,
                            "zoom": 10,
                            "isSplit": False
                        },
                        "mapStyle": {
                            "styleType": "dark",
                            "topLayerGroups": {},
                            "visibleLayerGroups": {
                                "label": True,
                                "road": True,
                                "border": False,
                                "building": True,
                                "water": True,
                                "land": True,
                                "3d building": False
                            },
                            "threeDBuildingColor": [
                                9.665468314072013,
                                17.18305478057247,
                                31.1442867897876
                            ],
                            "mapStyles": {}
                        }
                    }
                }

In [24]:
map_2 = KeplerGl(height=600, config=config)
map_2.add_data(tree_counts_lat_long[['lat', 'lng', 'value']], name='lat_longs')
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 's3afk3', 'type': '…